In [ ]:
! pip install openai==0.28

In [ ]:

models = openai.Model.list()

for model in models['data']:
    print(model['id'])

dall-e-2
whisper-1
gpt-3.5-turbo-instruct
tts-1-hd-1106
gpt-3.5-turbo
gpt-3.5-turbo-0125
babbage-002
davinci-002
dall-e-3
tts-1
gpt-3.5-turbo-16k
tts-1-1106
tts-1-hd
text-embedding-ada-002
text-embedding-3-small
text-embedding-3-large
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-3.5-turbo-1106
gpt-3.5-turbo-instruct-0914


In [ ]:
import openai
import os
import time

# OpenAI API 키 설정 (환경 변수에서 로드)
openai.api_key = os.getenv("OPENAI_API_KEY")

# 사용자 데이터 예시
user_data = {
    "age": 30,
    "gender": "female",
    "meal_time": "breakfast",
    "food_intake": {
        "rice": {"before": 200, "after": 100},
        "banana": {"before": 120, "after": 0},
        "kimchi": {"before": 50, "after": 20}
    },
    "nutrient_requirements": {
        "carbohydrates": 300,
        "proteins": 50,
        "fats": 70,
        "fiber": 25
    }
}

# 프롬프트 생성 함수
def create_prompt(data):
    prompt = f"연령대: {data['age']}세, 성별: {data['gender']}인 사용자의 {data['meal_time']}에 대한 피드백을 제공해주세요.\n\n"
    prompt += "사용자가 섭취한 음식과 전/후 섭취량:\n"
    for food, intake in data["food_intake"].items():
        consumed = intake["before"] - intake["after"]
        prompt += f"- {food}: {intake['before']}g 중 {consumed}g 섭취\n"

    prompt += "\n1인분 기준 권장 섭취량(일일 기준):\n"
    for nutrient, amount in data["nutrient_requirements"].items():
        prompt += f"- {nutrient}: {amount}g\n"

    prompt += "\n위 정보를 바탕으로 사용자의 섭취량이 적절한지 분석하고, 부족하거나 과다한 영양소에 대해 피드백을 제공해주세요. "
    prompt += "또한, 더 건강한 식습관을 위해 조언을 추가해 주세요."
    return prompt

# GPT API 요청 함수
def get_diet_feedback(data, retry_limit=3, delay=5):
    prompt = create_prompt(data)
    for attempt in range(retry_limit):
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a dietary feedback assistant."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=100,
                temperature=0.7
            )
            return response['choices'][0]['message']['content'].strip()
        except openai.error.RateLimitError:
            print(f"Rate limit reached. Retrying in {delay} seconds...")
            time.sleep(delay)
    return "할당량을 초과했습니다. 요금제 및 할당량을 확인해주세요."

# 맞춤형 피드백 생성 및 출력
feedback = get_diet_feedback(user_data)
print("맞춤형 피드백:\n", feedback)
